In [ ]:
__author__ = "Alina Molnar"
__copyright__ = "Copyright (C) 2020-2021 Alina Molnar"
__license__ = "CC BY-NC"
__version__ = "1.0"

# STEP 1. IMPORT LIBRARIES

In [ ]:
import glob
import os
from math import ceil
from pathlib import PureWindowsPath

import h2o
import IPython
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from h2o.estimators import (H2OGradientBoostingEstimator,
                            H2ORandomForestEstimator)
from IPython.display import display
from scipy.stats import spearmanr

## Settings

In [ ]:
# Set terminal to show all columns because I am interested in having an overview, not just the first few and the last few.
pd.options.display.max_rows = 99

# Do the same to show all rows because dataset is small.
pd.options.display.max_columns = 99

# Setting the colorpalette to "Colorblind" creates graphs accesible to everyone by removing red and green.
sns.set_palette("colorblind")

# Show info about system to help others reproduce the code.
print(IPython.sys_info())

plt.rcParams['figure.dpi'] = 100

# STEP 2. DATA UNDERSTANDING. CLEAN, TRANSFORM, PREPROCESS DATA

## 2.1 Collect Initial Data

In [ ]:
# Build the path to the following file in the repo: \beer_input\beer_241.xlsx
input_file = os.getcwd() + os.sep + "beer_input" + os.sep + "beer_241.xlsx"

# Read beer file
beer_all = pd.read_excel(input_file, sheet_name="Sheet1")

## 2.2 Describe Data

In [ ]:
# Write function that takes any dataframe and displays basic details about it.
# Use display() because it automatically adds an empty line between outputs, instead of print() which doesn't.

def show_basic_stats(df):
    """Display dataframe properties."""
    
    display(df.shape)
    display(df.info())
    display(df.describe().round(1))
    
show_basic_stats(beer_all)
# Rating is between -1 and 11.

### 2.2.1 Data Preprocessing: Standardization

In [ ]:
# Standardize appeareance. Convert column labels to lowercase.
beer_all.columns = beer_all.columns.str.lower()

# Convert columns values to lowercase if they are strings.
beer_all = beer_all.applymap(lambda col:col.lower() if type(col) == str else col)

# Convert Name column from object to string.
beer_all["name"] = beer_all["name"].astype("string")

# Cut alcohol content from end of name and store as separate column.
beer_all["abv"] = [name.rsplit(maxsplit=1)[-1] for name in beer_all["name"]]

# Convert alcohol content to float.
beer_all["abv"] = beer_all["abv"].astype(float)

# Convert object types to category, except Split column.
beer_all[["method", "style", "flavor", "fermentation"]] = beer_all[["method", "style", "flavor", "fermentation"]].astype("category")

### 2.2.2 Data Preprocessing: Pre-Validation

In [ ]:
# Validation of uniqueness in beer names. Check for duplicates, remove if found.
beer_all.drop_duplicates(subset="name", keep="last", inplace=True)

### 2.2.3 Data Preprocessing: Numeric Analysis

In [ ]:
# Description of numeric variables after standardized appearance and after removal of duplicates.
show_basic_stats(beer_all)

# At the beginning there were 241 rows and 8 columns, now there are 240 rows and 9 columns.

# Percentage of beer with ratings lower than 5.
under_5_rating = beer_all["rating"] < 5
under_5_rating_percentage = (len(beer_all[under_5_rating])/len(beer_all["rating"]))*100
print(f"Currently {under_5_rating_percentage:.1f}% of total beers are discarded.")

### 2.2.3 Data Preprocessing: Categorization

In [ ]:
# Description of categorical variables.

def describe_categorical_columns(df, numeric_column):
    """Print basic statistics of subgroups in categorical columns.
    
    Args:
    df (pandas.DataFrame): Dataframe containing numerical and categorical columns.
    numeric_column (str): Name of the column containing numerical data.

    Returns:
    Print basic statistics of each column containing categorical data: count, mean, std, min, 25%, 50%, 75%, max.
    """
    # Store in a list the columns containing categorical data.
    list_categoricals = df.select_dtypes(include=["category"]).columns.tolist()
    # Iterate through list of categorical columns.
    for i, elem in enumerate(list_categoricals):
        description = df.groupby([elem])[numeric_column].describe().round(1)
        print(description)

describe_categorical_columns(beer_all, "rating")

### 2.2.5 Data Preprocessing: String Columns

In [ ]:
# Description of Name column, string type, not categorical.
name_unique = len(set(beer_all["name"])) 
print(f"There are {name_unique} unique beer names.")

# Description of Country column, string type, not categorical.
country_unique = len(set(beer_all["country"])) 
print(f"There are {country_unique} unique countries.")

## 2.3 Explore Data


### 2.3.1 Hypothesis 1: There might be a linear relationship between ratings and alcohol content.

In [ ]:
# Plot alcohol content vs. rating by subgroup to check for hidden patterns.

def scatter_sns(df, x_numeric_column, y_numeric_column):
    """Seaborn scatter type subplots of two numeric variables as x and y, grouped by categorical columns.
    
    Args:
    df (pandas.DataFrame): Dataframe containing numerical and categorical columns.
    x_numeric_column (str): Name of numerical column to be plotted on x-axis.
    y_numeric_column (str): Name of numerical column to be plotted on y-axis.

    Returns:
    Seaborn scatter type subplots of x and y series, split by subgroups of categorical columns used as hue.
    Title and name of axes are added automatically based on the name of x and y series.
    """

    # Store in a list the columns containing categorical data.
    list_categoricals = df.select_dtypes(include=["category"]).columns.tolist()

    # Calculate the number of subplots in the figure.
    number_of_plots = len(list_categoricals)
    # Set the number of columns to 3 because it fits most screens.
    number_of_cols = 3
    # Calculate the number of rows in which subplots are shown.
    number_of_rows = ceil(number_of_plots/number_of_cols)

    # Plot figure and set title.
    fig = plt.figure()
    fig.suptitle(f"relationship between {df[x_numeric_column].name} and {df[y_numeric_column].name} by each categorical feature".title())
    
    # Iterate through list of categorical columns.
    for i, elem in enumerate(list_categoricals):
        # Add subplots sequentially.
        # Mark the first subplot as i+1 because subplot indices start at 1, and list indeces start at 0.
        ax = fig.add_subplot(number_of_rows, number_of_cols, i+1)
        
        # Create each subplot, set title of subplot, labels and legend.
        sns.scatterplot(x=df[x_numeric_column], y=df[y_numeric_column], hue=elem, data=df, s=15)
        ax.set_title(elem.title(), fontsize=12, verticalalignment="bottom", y=0.97)
        ax.set(xlabel=df[x_numeric_column].name.capitalize(), ylabel=df[y_numeric_column].name.capitalize())
        ax.legend(fontsize=5, loc="best")
        # In VS Code the legend is upper left in minimized window and in best location when maximized.      
    plt.show()

scatter_sns(beer_all, "abv", "rating")

# Result 1: The scatterplot shows no linear relationship and no pattern between ratings and ABV.
# However, subgroup of lemon flavor with zero or low ABV has higher ratings compared to other groups.

### 2.3.2 Hypothesis 2: Some subgroups might have a low number of observations.

In [ ]:
# Create column to store status of occurrences.
beer_all["occurrence"] = np.nan

# Count occurrences in subgroups of categorical data and return df with subgroups below threshold.
def select_too_few_categorical_observations(df, categorical_columns, occurrence_column, threshold_percentage):
    """Count observations of categorical features and store result in custom column.

    If subgroup has less observations than threshold, mark them as too_few in a results column.

    Args:
    df (pandas.DataFrame): Dataframe containing categorical columns.
    categorical_columns (list): List of categorical columns.
    occurrence_column (int): Name of column that stores the count of occurrences.
    threshold_percentage (int, float): Percentage of minimum observations from total.

    Returns:
    df (pandas.DataFrame): Selection from original dataframe.
    Rows contain subgroups with counted observations less than threshold.
    """

    # Calculate the number of rows needed to pass the threshold, and print result.
    threshold = len(df)*threshold_percentage/100
    print(f"The threshold is {threshold} observations or more.")

    # Iterate through list of categorical columns and count occurrences of subgroups.
    for i, elem in enumerate(categorical_columns):
        counted = df[elem].value_counts()

        # Convert counts to dictionary.
        counted_dictionary = counted.to_dict()
        # Iterate through dictionary and store result if too_few.
        for key, value in counted_dictionary.items():
            if value < threshold:
                df.loc[df[elem] == key, occurrence_column] = "too_few"
                print(f"Too few {key} {elem}.")

    # Fill occurrence column with "enough" if the record was not marked as too_few.
    for elem in df[occurrence_column]:
        if elem != "too_few":
            df.loc[df[occurrence_column] != "too_few", occurrence_column] = "enough"

    # Select rows with too_few observations.
    too_few = df.loc[df[occurrence_column] == "too_few"]
    return too_few

# Define list of categorical features to be checked and set a threshold of 5% from the total.
categoricals = ["method", "style", "flavor", "fermentation"]
too_few_subgroups = select_too_few_categorical_observations(beer_all, categoricals, "occurrence", 5)

# Result 2: Style and Flavor columns have subgroups below the 5% threshold of the total observations.

### Explain sequence for sorting bars of categoricals by value counts (numeric values) in the following function. Use subgroups of flavor column as example.

In [ ]:
# Start grouping all rows by flavor and select only the rating column, then calculate its mean.
# Reset the index to avoid future errors.
grouped_flavor = beer_all.groupby("flavor")["rating"].mean().reset_index()
# Then sort this series by ratings in descending order and select flavor labels.
ordered_flavor_mean = grouped_flavor.sort_values(by="rating", ascending=False)["flavor"]
# Lastly, turn these grouped and ordered flavor labels into a list ready to use when creating graphs.
stored_in_list_flavor_mean = list(ordered_flavor_mean)
# In a more concise (and hard to read) line, the flow looks like this:
desc_order_flavor_mean = list(beer_all.groupby("flavor")["rating"].mean().reset_index().sort_values(by="rating", ascending=False)["flavor"])

### 2.3.3 Hypothesis 3: Flavor column might have observable variation between its subgroups.

In [ ]:
# Plot bars of Flavor averages.
graph = sns.barplot(x="flavor", y="rating", data=beer_all, order=desc_order_flavor_mean)
graph.axhline(y=beer_all["rating"].mean(), linestyle="dashed", color="#C48170", label="feature mean")
graph.set(xlabel="Flavor", ylabel="Rating", title="Average Rating Of Style Subgroups")
graph.set_ylim([0, 10])
graph.legend()
plt.show()

# Result 3: Flavor subgroups have observable variation between their average.
# The errorbar is bigger on herb subgroup.

### 2.3.4 Hypothesis 4: Lemon beers’ high average might not be due to outliers.

In [ ]:
# Plot distribution of lemon flavored beer.
graph = sns.boxplot(x="flavor", y="rating", data=beer_all, order=desc_order_flavor_mean)
graph.axhline(y=beer_all["rating"].median(), linestyle="dashed", color="#C48170", label="feature median")
graph.set(xlabel="Flavor", ylabel="Rating", title="Distribution Of Ratings In Flavor Subgroups")
graph.set_ylim([0, 12])
graph.legend()
plt.show()

# Result 4: The boxplot shows that the distribution of lemon beers is due to higher ratings overall compared to other subgroups.
# There's no median on the lemon box. Let's find out why.

### Let's investigate what's going on with the lineless box of lemon ratings.

In [ ]:
# Check if lemon median equals one of the quantiles, remember median is 0.50 quantile.
lemon_ratings = beer_all[beer_all["flavor"] == "lemon"]["rating"]
print(lemon_ratings.quantile([0.25, 0.50, 0.75]))
# That's it, 0.50 and 0.75 quantile are equal, so that's why the unusual boxplot.
# Both 50% and 75% of all lemon ratings are higher or equal to 8.

# Check distribution of lemon beer ratings to see if the quantile explanation matches the graph.
graph = sns.kdeplot(x=lemon_ratings)
graph.set(title="KDE of Lemon Beer Rating", xlabel="Rating")
graph.set_xticks(range(-1, 12))
plt.show()
# There's a peak of observations where the rating is 8.
# More than half of the distribution is on the left side of the 8 mark, so 75% looks plausible.

# Conclusion: The ratings of lemon beers are so much higher than the rest, that their median overlaps with its 75th percentile.

## 2.4 Verify Data Quality

### 2.4.1 Data Coverage

#### 2.4.1.1 Data coverage in numerical variables

In [ ]:
# Data coverage in ratings.
unique_ratings = np.unique(beer_all["rating"])
unique_ratings_list = list(unique_ratings)
print(f"Uniques values of ratings are {unique_ratings_list}")

# Data coverage in abv.
unique_abv = np.unique(beer_all["abv"])
unique_abv_list = list(unique_abv)
print(f"Uniques values of alcohol content are {unique_abv_list}")

#### 2.4.1.2 Data coverage in categorical variables

In [ ]:
# How big are certain subgroups relative to the count of the others?

def countplot_sns(df):
    """Count of observations in each subgroup of categorical columns, plotted as bars.
    
    Args:
    df (pandas.DataFrame): Dataframe containing categorical columns.

    Returns:
    Seaborn countplot in subplots for co unting observations in each subgroup ofcategorical columns.
    Bars in descending order of counts.
    Axes labels are added automatically based on column names.
    """

    # Select columns with data type category.
    list_categoricals = df.select_dtypes(include=["category"]).columns.tolist()
    # Calculate the number of subplots in the figure.
    number_of_plots = len(list_categoricals)
    # Set the number of columns to 3 because it fits most screens.
    number_of_cols = 3
    # Calculate the number of rows in which subplots are shown.
    number_of_rows = ceil(number_of_plots/number_of_cols)

    # Plot figure and set title.
    fig = plt.figure()
    fig.suptitle("count of observations in each subgroup of categorical columns".title())
    # Iterate through list of categorical columns.
    for i, elem in enumerate(list_categoricals):
        # Define bar sorting criteria as descending counts.
        desc_order = df[elem].value_counts().index
        # Add subplots sequentially.
        # Mark the first subplot as i+1 because subplot indices start at 1, and list indeces start at 0.
        ax = fig.add_subplot(number_of_rows, number_of_cols, i+1)
        
        # Create each subplot, set labels and legend.
        sns.countplot(x=elem, data=df, order=desc_order)
        ax.set_title(elem.title(), fontsize=12, verticalalignment="bottom", y=0.97)
        ax.set_xticklabels(desc_order, fontsize=6, rotation=30, horizontalalignment="right", verticalalignment="top")
        ax.set(xlabel="", ylabel="")
    plt.show()

countplot_sns(beer_all)
# Size of subgroups consistent with stores' assortment.

# STEP 3. DATA PREPARATION

## 3.1 Select Data

### 3.1.1 Distribution of ratings

In [ ]:
graph = sns.kdeplot(x=beer_all["rating"])
graph.set(title="KDE Of Rating", xlabel="Rating")
graph.set_xticks(range(-1, 12))
plt.show()
# Curve of ratings KDE is gaussian.

### 3.1.2 Distribution of alcohol content

In [ ]:
# Commercial beer is either regular, alcohol-free, or lemonade mix, and a smooth curve hides these groups.
# Set bandwidth lower than 1 to check if groups show up.  
graph = sns.kdeplot(x=beer_all["abv"], label="smoothed curve")
graph = sns.kdeplot(x=beer_all["abv"], bw_adjust=0.3, label="focused curve")
graph.set(title="KDE Of Alcohol Content", xlabel="Alcohol content")
graph.set_xticks(range(0, 11))
graph.legend()
plt.show()
# Curve of alcohol content KDE is not gaussian.
# Still, it shows a pattern of three subgroups each with its own gaussian curve.

# Check proportion of alcohol-free beer because it influences the curve of alcohol content.
abv_list = beer_all["abv"].tolist()
abv_zero = (abv_list.count(0)/len(abv_list))*100
print(f"Alcohol-free are {abv_zero:.2f}% of total beer.")

### 3.1.3 Combined distribution of alcohol content and ratings

In [ ]:
# Calculate Spearman's correlation coefficient because it works for non-linear relationship if the variables are monotonic.
spearman_corr, _ = spearmanr(beer_all["abv"], beer_all["rating"])
print(f"Spearman\'s correlation: {spearman_corr:.2f}.")
# Result: -0.11, so if they have any kind of relationship, it is not monotonic.
# The scatterplot between abv and rating is consistent with Spearman's correlation coefficient.

# Plot KDE of alcohol content and rating. Set bandwidth less than 1 because distribution of ABV is not gaussian.
graph = sns.kdeplot(data=beer_all, x=beer_all["abv"], y=beer_all["rating"], bw_adjust=0.7, color="#C48170")
graph.set(title="KDE of alcohol content and rating", xlabel="Alcohol content", ylabel="Rating")
plt.show()
# There are three zones, so it makes sense to split the dataset into three groups after all cleanup is done.

### 3.1.4 Check country column if it has enough observations for each unique value, otherwise drop the column

In [ ]:
# Count occurrences for each country.
country_count = beer_all["country"].value_counts()

# Count occurrences for each country as percentage.
country_percentage = beer_all["country"].value_counts(normalize=True).mul(100).round(1)

# Collect all results in one dataframe.
country_stats = pd.DataFrame({"observations": country_count, "percentage": country_percentage})
print(country_stats)
# There are 17 unique countries and 15 of them have each less than 5% of the total observations.

# Drop country column.
beer_all = beer_all.drop("country", 1)

### 3.1.5 Check which categorical features have high variation across subgroups for building models

In [ ]:
# Plot ratings variation in subgroups across categorical features.

def barplot_sns(df, y_numeric_column, graph_title, categorical_columns=None):
    """Seaborn subplots of bars showing mean of numeric column grouped by categorical columns.
    If list of categorical columns is not provided, uses columns of category type from dataframe.
    
    Args:
    df (pandas.DataFrame): Dataframe containing numerical and categorical columns.
    y_series (str): Name of numerical column.
    graph_title (str): Title of graph.
    categorical_columns (list, optional): List of categorical columns.

    Returns:
    Seaborn subplots of bars with mean of each subgroup in categorical columns.
    Horizontal line to mark the mean of each column. Bars in descending order of mean.
    Axes labels are added automatically based on column names.
    """
    # Check if input contains list of categorical columns. If not, select columns with data type category.
    if categorical_columns != None:
        list_categoricals = categorical_columns
    else:
        list_categoricals = df.select_dtypes(include=["category"]).columns.tolist()
    
    # Calculate the number of subplots in the figure.
    number_of_plots = len(list_categoricals)
    # Set the number of columns to 3 because it fits most screens.
    number_of_cols = 3
    # Calculate the number of rows in which subplots are shown.
    number_of_rows = ceil(number_of_plots/number_of_cols)

    # Plot figure and set title.
    fig = plt.figure()
    fig.suptitle(graph_title.title())

    # Iterate through list of categorical columns.
    for i, elem in enumerate(list_categoricals):
    # Define bar sorting criteria as descending mean.
        desc_order = list(df.groupby(elem)[y_numeric_column].mean().reset_index().sort_values(by=y_numeric_column, ascending=False)[elem])
        # Add subplots sequentially.
        # Mark the first subplot as i+1 because subplot indices start at 1, and list indeces start at 0.
        ax = fig.add_subplot(number_of_rows, number_of_cols, i+1)
        # Create each subplot, set horizontal line to mark the mean, set labels and legend.
        sns.barplot(x=elem, y=y_numeric_column, data=df, order=desc_order, dodge=False)
        ax.axhline(y=df[y_numeric_column].mean(), linestyle="dashed", color="#C48170", label="feature mean")
        ax.set_title(elem.title(), fontsize=12, verticalalignment="bottom", y=0.97)
        ax.set_xticklabels(desc_order, fontsize=6, rotation=30, horizontalalignment="right", verticalalignment="top")
        ax.set(xlabel="", ylabel="")
        ax.set_ylim([0, 10])
        ax.legend()
        
    plt.show()
 
barplot_sns(beer_all, "rating", "average rating of beer subgroups", categoricals)


# The barplot shows Flavor and Style have high variation between the average of their subgroups.
# Method and Fermentation have low variation across their subgroups.
# Errorbars are bigger on subgroups with low number of observations.

In [ ]:
# Abstract representation of ratings distribution. Take a look at median, IQR, whiskers, outliers.

def boxplot_sns(df, y_numeric_column, graph_title, categorical_columns=None):
    """Seaborn subplots with boxplots of subgroups' distribution in categorical columns.
    If list of categorical columns is not provided, uses columns of category type from dataframe.
    
    Args:
    df (pandas.DataFrame): Dataframe containing numerical and categorical columns.
    y_series (str): Name of numerical column.
    graph_title (str): Title for graph.
    categorical_columns (list, optional): List of categorical columns.

    Returns:
    Seaborn subplots of boxplots showing distribution of subgroups in categorical columns.
    Boxes in descending order of median.
    Axes labels are added automatically based on column names.
    """

    # Check if input contains list of categorical columns. If not, select columns with data type category.
    if categorical_columns != None:
        list_categoricals = categorical_columns
    else:
        list_categoricals = df.select_dtypes(include=["category"]).columns.tolist()
    
    # Calculate the number of subplots in the figure.
    number_of_plots = len(list_categoricals)
    # Set the number of columns to 3 because it fits most screens.
    number_of_cols = 3
    # Calculate the number of rows in which subplots are shown.
    number_of_rows = ceil(number_of_plots/number_of_cols)
    
    # Plot figure and set title.
    fig = plt.figure()
    fig.suptitle(graph_title.title())
    # Iterate through list of categorical columns.
    for i, elem in enumerate(list_categoricals):
        # Define box sorting criteria as descending median.
        desc_order = list(df.groupby(elem)[y_numeric_column].median().reset_index().sort_values(by=y_numeric_column, ascending=False)[elem])
        # Add subplots sequentially.
        # Mark the first subplot as i+1 because subplot indices start at 1, and list indeces start at 0.
        ax = fig.add_subplot(number_of_rows, number_of_cols, i+1)

        # Create each subplot, set labels and legend.
        sns.boxplot(x=elem, y=y_numeric_column, data=df, order=desc_order)
        ax.axhline(y=df[y_numeric_column].median(), linestyle="dashed", color="#C48170", label="feature median")
        ax.set_title(elem.title(), fontsize=12, verticalalignment="bottom", y=0.97)
        ax.set_xticklabels(desc_order, fontsize=6, rotation=30, horizontalalignment="right", verticalalignment="top")
        ax.set(xlabel="", ylabel="")
        ax.set_ylim([0, 12])
        ax.legend()

    plt.show()

boxplot_sns(beer_all, "rating", "Rating distribution of beer subgroups", categoricals)

# The boxplot shows Flavor and Style have high variation between the distributions of their subgroups.
# Method and Fermentation have low variation across their subgroups.
# Errorbars are bigger on subgroups with low number of observations.

In [ ]:
# Plot distribution of style subgroups to have an individual image of Style for the powerpoint presentation.
desc_order_style = list(beer_all.groupby("style")["rating"].median().reset_index().sort_values(by="rating", ascending=False)["style"])
graph = sns.boxplot(x="style", y="rating", data=beer_all, order=desc_order_style)
graph.axhline(y=beer_all["rating"].median(), linestyle="dashed", color="#C48170", label="feature median")
graph.set(xlabel="Style", ylabel="Rating", title="Distribution Of Ratings In Style Subgroups")
graph.set_ylim([0, 12])
graph.legend()
plt.show()

## 3.2 Clean data

In [ ]:
# Define range of normal ratings as 2*std away from the mean because dataset is small and has gaussian distribution.
mean_rating = beer_all["rating"].mean()
std_rating = beer_all["rating"].std()
two_std_rating = std_rating * 2

lower_limit_rating = mean_rating - two_std_rating
upper_limit_rating = mean_rating + two_std_rating

print(f"The mean rating is {mean_rating:.2f}.")
print(f"The lower limit of normal ratings is {lower_limit_rating:.2f} and the upper limit is {upper_limit_rating:.2f}.")

# Identify rating outliers. Use result to train machine learning model and avoid overfitting.
outlier_ratings = [x for x in beer_all["rating"] if x < lower_limit_rating or x > upper_limit_rating]
outlier_ratings.sort()
# print(f"These are the rating outliers: {outlier_ratings}")
print(f"There are {len(outlier_ratings)} outliers out of {len(beer_all.rating)} total rating observations.")

## 3.3 Construct Data

### 3.3.1 Derived Attributes

In [ ]:
# Create column for filtration status.
unfiltered_words = ["unfiltered", "kellerbier", "natur", "naturtrubes", "nefiltrata", "nonfiltrata"]
beer_all["filtration"] = beer_all["name"].str.contains("|".join(unfiltered_words))
beer_all["filtration"] = beer_all["filtration"].replace({True: "unfiltered", False: "filtered"})

# Create column for pasteurization status.
unpasteurized_words = ["unpasteurized", "kellerbier", "natur", "naturtrubes", "nepasteurizata", "nonpastorizzata"]
beer_all["pasteurization"] = beer_all["name"].str.contains("|".join(unpasteurized_words))
beer_all["pasteurization"] = beer_all["pasteurization"].replace({True: "unpasteurized", False: "pasteurized"})

# Format the new columns as categoricals.
beer_all[["filtration", "pasteurization"]] = beer_all[["filtration", "pasteurization"]].astype("category")

### 3.3.2 Generated records

In [ ]:
# Create column to bin alcohol content as categorical data.
abv_bins = [0, 0.5, 2.8, 4.4, 5.5, 10]
perception_labels = ["drive", "refresh", "weak", "tasty", "too_strong"]
beer_all["perception"] = pd.cut(beer_all["abv"], bins=abv_bins, labels=perception_labels, include_lowest=True)

## 3.4 Integrate Data

### 3.4.1 Select observations with ratings less than 2*std away from the mean

In [ ]:
# Use limits calculated at 3.2 and reset index.
outlier_condition = (beer_all.rating < lower_limit_rating) | (beer_all.rating > upper_limit_rating)
beer_2std = beer_all.drop(beer_all[outlier_condition].index)
beer_2std.reset_index(inplace=True, drop=True)
print(f"Dataframe without outliers has {len(beer_2std.rating)} rows.")

# Check this out, below selection by square brackets doesn't work in f-string, must use dot notation.
# print(f"There are {len(beer_2std["rating"])} observations with normal ratings.")

### 3.4.2 Split dataset by alcohol content

In [ ]:
# Split dataset by alcohol content into three subsets: alcohol-free, light and regular beer.
alc_free_all = beer_all[beer_all["abv"] <= 0.5]
light_all = beer_all[(beer_all["abv"] > 0.5) & (beer_all["abv"] <= 3)]
regular_all = beer_all[beer_all["abv"] > 3]

alc_free_2std = beer_2std[beer_2std["abv"] <= 0.5]
light_2std = beer_2std[(beer_2std["abv"] > 0.5) & (beer_2std["abv"] <= 3)]
regular_2std = beer_2std[beer_2std["abv"] > 3]

print(f"In complete ratings range there are {len(alc_free_all)} alcohol-free, {len(light_all)} light and {len(regular_all)} regular beers.")
print(f"In less than 2*std away ratings there are {len(alc_free_2std)} alcohol-free, {len(light_2std)} light and {len(regular_2std)} regular beers.")

### 3.4.3 Check distribution of ratings in subsets

In [ ]:
# Plot distribution of ratings in each subset to check if their curves are gaussian.
graph = sns.kdeplot(x=alc_free_all["rating"], label="all")
graph = sns.kdeplot(x=alc_free_2std["rating"], label="2_std")
graph.set(title="Alcohol-free Beer Ratings", xlabel="Rating")
graph.set_xticks(range(-1, 12))
plt.legend()
plt.show()

graph = sns.kdeplot(x=light_all["rating"], label="all")
graph = sns.kdeplot(x=light_2std["rating"], label="2_std")
graph.set(title="Light Beer Ratings", xlabel="Rating")
graph.set_xticks(range(-1, 12))
plt.legend()
plt.show()

graph = sns.kdeplot(x=regular_all["rating"], label="all")
graph = sns.kdeplot(x=regular_2std["rating"], label="2_std")
graph.set(title="Regular Beer Ratings", xlabel="Rating")
graph.set_xticks(range(-1, 12))
plt.legend()
plt.show()
# KDE plots of the six subsets prove that ratings keep their gaussian curve even if split by alcohol content criteria.

In [ ]:
# Create a dictionary that stores datasets as values and their names as keys.
clean_dataframes = [beer_all, alc_free_all, light_all, regular_all, beer_2std, alc_free_2std, light_2std, regular_2std]
dataframe_names = ["beer_all", "alc_free_all", "light_all", "regular_all", "beer_2std", "alc_free_2std", "light_2std", "regular_2std"]

dataframes_dict = dict(zip(dataframe_names, clean_dataframes))

### 3.4.4 Feature variation across subgroups of alcohol content

In [ ]:
# Check feature variation across subgroups when split into the three subsets based on alcohol content.

for key, value in dataframes_dict.items():
    boxplot_sns(value, "rating", f"Distribution of {key}")
    plt.show()

# All subsets have higher variation in Style and Flavor, and no variation in Method.
# Some subsets have a bit of variation in Filtration, Pasteurization, Fermentation and Perception.

## 3.5 Format Data

In [ ]:
# Formatting was done in previous preprocessing tasks, and the only task left is sorting.
# Sort dataframe on ABV, then on rating.
beer_all.sort_values(["rating", "abv"], ascending=[True, True], inplace=True)

## 3.6 Dataset - Output

In [ ]:
# Export dataframes and remove index because beer identification is done through their unique names.
# It's also good to avoid having two columns with indices next time the file is imported.


# Create function to export dictionary of dataframes to csv files.

def export_dict_as_csv(dataframes_dictionary, output_folder, suffix=None):
    """Export dictionary as csv files. Dictionary contains names as keys and dataframes as values.
    
    Args:
    dataframes_dictionary (dict): dictionary of dataframes as values and their names as keys.
    output_folder (str): path where to export files.
    suffix (str, optional): suffix to add after filename.

    Returns:
    csv files
    """

    for key, value in dataframes_dictionary.items():
        # Extract filename.
        filename = str(key)

        # Create file path.
        if suffix != None:
            output_address = output_folder + os.sep + filename + suffix + ".csv"
        else:
            output_address = output_folder + os.sep + filename + ".csv"

        # Export dataframe.
        value.to_csv(output_address, index=False)


# Path to folder containing clean files.
clean_files_path = os.getcwd() + os.sep + "beer_output" + os.sep + "clean_files"

# Export clean files.
export_dict_as_csv(dataframes_dict, clean_files_path)

## 3.7 Dataset Description

In [ ]:
# Print total number of columns, rows and ratings lower than 5.
# Print description of categorical columns.

for key, value in dataframes_dict.items():
    total_rows = value.shape[0]
    lower_than_five = value[value["rating"] < 5]
    only_lows = lower_than_five.shape[0]
    total_columns = value.shape[1]
    print(f"{key} \nRows: {total_rows} \nNumber of ratings lower than 5: {only_lows} \nColumns: {total_columns} \n")
    describe_categorical_columns(value, "rating")

# STEP 4. MODELING. MACHINE LEARNING WITH H2O

### 4.1 Initialize H2O

In [ ]:
# Assertions are disabled because they are mainly used for error checking and debugging purposes.
# nthreads=-1 means use all CPU on the host.
h2o.init(nthreads=-1, enable_assertions=False)

### 4.2 Select .csv Files

In [ ]:
# Select clean files.
clean_files = glob.glob(clean_files_path + os.sep + "*.csv")

### 4.3 Import Datasets As Dictionaries Into H2O

In [ ]:
# Write function to import multiple files at once.

def files_to_h2o_frames(files, response_column):
    """Import files into H2O frames.
    
    Args:
    files (list): List of files to be imported.
    response_column (str): Name of response column.

    Returns:
    dict: Dictionary containing H2O frames, predictors and response. Key: the name of a dataset. Values: the imported frame, its list of predictors, the response column.
    """

    # Create empty dictionary to be populated at each iteration.
    dictionary = {}

    # Iterate through list of files to create name, frame and list of predictors.
    for i, elem in enumerate(files):
        name = PureWindowsPath(elem).stem
        frame = h2o.import_file(elem)
        
        if "alc_free" in name:
            predictors = ["style", "flavor"]
        elif "light" in name:
            predictors = ["style", "flavor", "abv"]
        elif "regular" in name:
            predictors = ["style", "flavor", "pasteurization", "abv"]
        else:
            predictors = ["style", "flavor", "perception", "abv"]

        # Add key and values to dictionary.
        dictionary[name] = {"frame":frame, "predictors":predictors, "response": response_column}
    return dictionary

frames_dictionary = files_to_h2o_frames(clean_files, "rating")

### 4.4 Select Modeling Techniques

In [ ]:
# I have selected Distributed Random Forest (DRF), Gradient Boosting Machine (GBM) - see motives in Final Report file.

### 4.5 Generate Test Design

In [ ]:
# The model will learn from the training set and will be assessed on the test set.
# Train 0.7, valid 0.15 and test 0.15 splits were decided manually to ensure they are diverse no matter how few observations there are.
# See note in Readme file.

### 4.6 Build Models

### 4.6.1 Parameter Settings

In [ ]:
# Parameters are listed in the Final Report, together with an explanation fow why I have chosen them.
# Parameters are grouped by model type DRF and GBM, and for each model they are grouped by instantiate, training, testing, and export tasks. 

### 4.6.2 Define Models

#### 4.6.2.1 Distributed Random Forest - DRF

In [ ]:
# Write function to generate DRF model and export prediction as pandas dataframe.

def model_h2o_drf(frames, model_output_folder, pred_output_folder, mse_output_folder):
    """Build DRF model in H2O for each dataset, add prediction to pandas dataframe, export result and MSE as csv.

    Args:
    frames (dict): Dictionary containing frames, predictors and response column.
    model_output_folder (str): Path to folder where to export DRF model.
    pred_output_folder (str): Path to folder where to export predictions from DRF model.
    mse_output_folder (str): Path to folder where to export file with MSE of all DRF models.

    Returns:
    zip archive of each model
    csv file with predictions of each model
    csv file with MSE of all DRF models
    """

    # Create list of model names.
    model_names = []

    # Create list of MSE results from each model.
    mse_list = []

    # Iterate through dictionary and store its elements under short variable names to help with readability.
    for key, value in frames.items():
        # Access the name.
        name = str(key)+ "_drf"

        # Append model name to list.
        model_names.append(name)

        # Access the frame.
        frame = frames[key]["frame"]
        # Access the predictors.
        predictor_list = frames[key]["predictors"]
        # Access the response.
        response_column = frames[key]["response"]

        # Split rows into training, validation and test sets. This makes reproducibility possible.
        train = frame[frame["split"]=="train"]
        valid = frame[frame["split"]=="valid"]
        test = frame[frame["split"]=="test"]

        # Instantiate model with custom parameters explained in Final Report, step 4.6.1.
        model = H2ORandomForestEstimator(seed=12, categorical_encoding="Enum", nfolds=4, fold_assignment="random", 
        mtries=len(predictor_list), nbins=13, nbins_top_level=16, build_tree_one_node=True)

        # Train model. Specify predictors, response column, training frame and validation frame.
        model.train(x=predictor_list, y=response_column, training_frame=train, validation_frame=valid, model_id=name+"_model")

        # Select variable importance from model json.
        model_output = model._model_json["output"]
        var_imp_values = model_output["variable_importances"].cell_values
        var_imp_header = model_output["variable_importances"].col_header
        variable_importance = pd.DataFrame(var_imp_values, columns=var_imp_header)

        # Generate prediction. It gets stored in a H2O frame with one column named "predict".
        prediction = model.predict(frame)

        # Calculate model performance on test set.
        performance = model.model_performance(test)

        # Store model performance as json into a dictionary.
        perf_dict = performance._metric_json

        # Select only MSE from performance dictionary. Use ndarray.item method to catch errors in case MSE output is not a float.
        mse_value = np.asarray([value for key, value in perf_dict.items() if key == "MSE"]).item()
        
        # Append MSE list.
        mse_list.append(mse_value)

        # Add prediction to original H2O frame to help further analysis.
        dataset_plus_prediction = frame.cbind(prediction)

        # Convert H2O predictions frame to pandas dataframe.
        dataset_plus_prediction_pandas = dataset_plus_prediction.as_data_frame()

        # Export prediction dataframe.
        dataset_plus_prediction_pandas.to_csv(pred_output_folder + os.sep + name + ".csv", index=False)

        # Export model.
        model.download_mojo(path=model_output_folder + os.sep + name + ".zip", get_genmodel_jar=False)

        # Export variable importance.
        variable_importance.to_csv(model_output_folder + os.sep + name + "_varimp.csv", index=False)
        

    # Zip names and MSE values into a pandas dataframe.
    mse_models = pd.DataFrame(zip(model_names, mse_list), columns=["model_name", "mse"])

    # Export MSE dataframe to csv file.
    mse_models.to_csv(mse_output_folder + os.sep + "drf_mse.csv", index=False)

#### 4.6.2.2 Gradient Boosting Machine - GBM

In [ ]:
# Write function to generate GBM model and export prediction as pandas dataframe.

def model_h2o_gbm(frames, model_output_folder, pred_output_folder, mse_output_folder):
    """Build GBM model in H2O for each dataset, add prediction to pandas dataframe, export result and MSE as csv.

    Args:
    frames (dict): Dictionary containing frames, predictors and response column.
    model_output_folder (str): Path to folder where to export GBM model.
    pred_output_folder (str): Path to folder where to export predictions from GBM model.
    mse_output_folder (str): Path to folder where to export file with MSE of all GBM models.

    Returns:
    zip archive of each model
    csv file with predictions of each model
    csv file with MSE of all GBM models
    """
    
    # Create list of model names.
    model_names = []

    # Create list of MSE results from each model.
    mse_list = []

    # Iterate through dictionary and store its elements under short variable names to help with readability.
    for key, value in frames.items():
        # Access the name.
        name = str(key) + "_gbm"

        # Append model name to list.
        model_names.append(name)

        # Access the frame.
        frame = frames[key]["frame"]
        # Access the predictors.
        predictor_list = frames[key]["predictors"]
        # Access the response.
        response_column = frames[key]["response"]

        # Split rows into training, validation and test sets. This makes reproducibility possible.
        train = frame[frame["split"]=="train"]
        valid = frame[frame["split"]=="valid"]
        test = frame[frame["split"]=="test"]

        # Instantiate model with custom parameters explained in Final Report, step 4.6.1.
        model = H2OGradientBoostingEstimator(seed=12, categorical_encoding="Enum", nfolds=4, fold_assignment="random", 
        min_rows=1, nbins=13, nbins_top_level=16, distribution="gaussian", build_tree_one_node=True)

        # Train model. Specify predictors, response column, training frame and validation frame.
        model.train(x=predictor_list, y=response_column, training_frame=train, validation_frame=valid, model_id=name+"_model")

        # Select variable importance from model json.
        model_output = model._model_json["output"]
        var_imp_values = model_output["variable_importances"].cell_values
        var_imp_header = model_output["variable_importances"].col_header
        variable_importance = pd.DataFrame(var_imp_values, columns=var_imp_header)

        # Generate prediction. It gets stored in a H2O frame with one column named "predict".
        prediction = model.predict(frame)

        # Calculate model performance on test.
        performance = model.model_performance(test)

        # Store model performance as json into a dictionary.
        perf_dict = performance._metric_json

        # Select only MSE from performance dictionary. Use ndarray.item method to catch errors in case MSE output is not a float.
        mse_value = np.asarray([value for key, value in perf_dict.items() if key == "MSE"]).item()
        
        # Append MSE list.
        mse_list.append(mse_value)

        # Add prediction to original H2O frame to help further analysis.
        dataset_plus_prediction = frame.cbind(prediction)

        # Convert H2O predictions frame to pandas dataframe.
        dataset_plus_prediction_pandas = dataset_plus_prediction.as_data_frame()

        # Export prediction dataframe.
        dataset_plus_prediction_pandas.to_csv(pred_output_folder + os.sep + name + ".csv", index=False)
        
        # Export model.
        model.download_mojo(path=model_output_folder + os.sep + name + ".zip", get_genmodel_jar=False)

        # Export variable importance.
        variable_importance.to_csv(model_output_folder + os.sep + name + "_varimp.csv", index=False)
        
    # Zip names and MSE values into a pandas dataframe.
    mse_models = pd.DataFrame(zip(model_names, mse_list), columns=["model_name", "mse"])

    # Export MSE dataframe to csv file.
    mse_models.to_csv(mse_output_folder + os.sep + "gbm_mse.csv", index=False)

### 4.6.3 Call Functions to Build Models

In [ ]:
# Output folders for models.
model_drf_folder = os.getcwd() + os.sep + "beer_output" + os.sep + "models" + os.sep + "drf_models"
model_gbm_folder = os.getcwd() + os.sep + "beer_output" + os.sep + "models" + os.sep + "gbm_models"

# Output folders for predictions dataframes resulted from DRF and GBM models.
pred_drf_folder = os.getcwd() + os.sep + "beer_output" + os.sep + "predictions" + os.sep + "drf_predictions"
pred_gbm_folder = os.getcwd() + os.sep + "beer_output" + os.sep + "predictions" + os.sep + "gbm_predictions"

# Output folder for MSE of models.
mse_folder = os.getcwd() + os.sep + "beer_output" + os.sep + "metrics" + os.sep + "mse"

# Call functions that build models and export dataframes. 
model_h2o_drf(frames_dictionary, model_drf_folder, pred_drf_folder, mse_folder)
model_h2o_gbm(frames_dictionary, model_gbm_folder, pred_gbm_folder, mse_folder)

### 4.7 Assess Models

In [ ]:
# Create function to select files with a certain pattern.

def import_csv_files_as_dict(files_path, files_pattern, check_subfolders):
    """Import csv files that match a pattern, return dictionary with filenames as keys and dataframes as values.
    Args:
    files_path (str):
    files_pattern (str):
    check_subfolders (bool): True if imports from subfolders. False if imports only from main folder.

    Returns:
    names_dataframes_dict (dict): dictionary of dataframes as values and their names as keys.
    """
    
    # Select files that match pattern. Recursive parameter decides if selection contains files from subfolders.
    files = glob.glob(files_path + os.sep + files_pattern, recursive=check_subfolders)

    # Extract name of files and use them to store dataframes.
    names = [PureWindowsPath(elem).stem for elem in files]

    # Read files into pandas dataframes.
    dataframes = [pd.read_csv(item) for item in files]

    # Create dictionary to store names and dataframes of predictions.
    names_dataframes_dict = dict(zip(names, dataframes))

    # Return dictionary of names and dataframes.
    return names_dataframes_dict

In [ ]:
# See discussion in Final Report about built-in metrics of models.
# Import variable importance files.
var_imp_path = os.getcwd() + os.sep + "beer_output" + os.sep + "models"
var_imp_pattern = "/**/*.csv"
var_imp_dict = import_csv_files_as_dict(var_imp_path, var_imp_pattern, True)

# Print variable importance dataframes.
for key, value in var_imp_dict.items():
    print(key, value, "\n")

# Most important predictors are Flavor in alc_free_drf_all, ABV in light_all_drf, Style in regular_all_drf and beer_all_drf.

# STEP 5. EVALUATION

## 5.1 Evaluate results

### 5.1.1 Assessment of data mining results w.r.t. business success criteria

In [ ]:
# Steps for evaluating model real-life performance:
# 1. Import predictions.
# 2. Select false negatives, which means ratings lower than 5 that were predicted as 5 or higher.
# 3. Calculate recall score.
# 4. Approve models that have the highest recall score.

#### 5.1.1.1 Import predictions dataframes

In [ ]:
# Import predictions dataframes generated by models.
predictions_path = os.getcwd() + os.sep + "beer_output" + os.sep + "predictions"
predictions_pattern = "/**/*.csv"

predictions_dict = import_csv_files_as_dict(predictions_path, predictions_pattern, True)

#### 5.1.1.2 Select false negatives

In [ ]:
# Create function to generate dictionary of false negatives dataframes.

def select_false_negatives(names_dataframes_dict, real_response_column, predict_column, threshold):
    """Create false negatives dictionary, names as keys and dataframes as values.
    The name of the response column should be the same in all dataframes. Same applies for predicted column.

    Args:
    names_dataframes_dict (dict): Dictionary of dataframes and their names.
    real_response_column (str): The name of the response column with numerical data as values.
    predict_column(str): The name of the predicted column with numerical data as values.
    threshold (int, float): The threshold that separates outcomes.

    Returns:
    false_negatives_dict (dict): dictionary of false negatives dataframes as values and their names as keys.
    """

    # Create empty dictionary to store false negatives.
    false_negatives_dict = {}

    # Iterate through input dictionary.
    for key, value in names_dataframes_dict.items():
        
        # Create name of false negatives dataframe.
        name = str(key) + "_fn"

        # Access the dataframe stored as value in input dictionary.
        dataframe = names_dataframes_dict[key]

        # Select true positives and false negatives:
        false_negatives = dataframe[(dataframe[real_response_column] < threshold) & (dataframe[predict_column] >= threshold)]

        # If false negatives dataframe is not empty, append it to the false negatives dictionary.
        if not false_negatives.empty:
            false_negatives_dict[name] = false_negatives
        else:
            pass
    
    # Return dictionary of false negatives.
    return false_negatives_dict


false_negatives = select_false_negatives(predictions_dict, "rating", "predict", 5)

In [ ]:
# Export false negatives.
false_negatives_path = os.getcwd() + os.sep + "beer_output" + os.sep + "false_negatives"
export_dict_as_csv(false_negatives, false_negatives_path)

#### 5.1.1.3 Calculate recall score

In [ ]:
# Create function to calculate recall score.

def calculate_recall_score(names_dataframes_dict, real_response_column, predict_column, threshold):
    """Calculate recall score of models by counting true positives and false negatives in their predictions.

    Args:
    names_dataframes_dict (dict): Dictionary of dataframes and their names.
    real_response_column (str): The name of the response column with numerical data as values.
    predict_column(str): The name of the predicted column with numerical data as values.
    threshold (int, float): The threshold that separates outcomes.
    
    Returns:
    recall_dataframe (pandas.DataFrame): dataframe of model names and their recall scores.
    """

    # Create empty dictionary to store recall scores.
    recall_dict = {}
    
    # Iterate through input dictionary.
    for key, value in names_dataframes_dict.items():

        # Access the name of the model.
        name = str(key)
        
        # Access the dataframe stored as value in input dictionary.
        dataframe = names_dataframes_dict[key]

        # Select true positives and false negatives:
        true_positives = dataframe[(dataframe[real_response_column] < threshold) & (dataframe[predict_column] < threshold)]
        false_negatives = dataframe[(dataframe[real_response_column] < threshold) & (dataframe[predict_column] >= threshold)]

        # Calculate recall score, where df.shape[0] counts the rows of a df.
        try:
            recall = true_positives.shape[0] / (false_negatives.shape[0] + true_positives.shape[0])
        except ZeroDivisionError:
            recall = np.NaN    

        # Append score to recall dictionary.
        recall_dict[name] = recall

    # Convert dictionaty to dataframe.
    recall_dataframe = pd.DataFrame(list(recall_dict.items()), columns=["model_name", "recall_score"])

    # Return recall score dataframe.
    return recall_dataframe


recall = calculate_recall_score(predictions_dict, "rating", "predict", 5)

In [ ]:
# Export recall dataframe.
recall_path = os.getcwd() + os.sep + "beer_output" + os.sep + "metrics" + os.sep + "recall" + os.sep + "recall_score.csv"
recall.to_csv(recall_path, index=False)

#### 5.1.1.4 Merge MSE and recall in a single dataframe

In [ ]:
# Import MSE files.
drf_mse = pd.read_csv(os.getcwd() + os.sep + "beer_output" + os.sep + "metrics" + os.sep + "mse" + os.sep + "drf_mse.csv")
gbm_mse = pd.read_csv(os.getcwd() + os.sep + "beer_output" + os.sep + "metrics" + os.sep + "mse" + os.sep + "gbm_mse.csv")

# Merge MSE files.
merged_mse = drf_mse.merge(gbm_mse, how="outer")

# Merge total MSE with recall score.
mse_recall = merged_mse.merge(recall, how="outer")

In [ ]:
# Create column to store the model type.
mse_recall["model_type"] = mse_recall["model_name"].str.contains("drf")
mse_recall["model_type"] = mse_recall["model_type"].replace({True: "DRF", False: "GBM"})

# Create a column to store the range of each dataset.
mse_recall["dataset_range"] = mse_recall["model_name"].str.contains("_2std")
mse_recall["dataset_range"] = mse_recall["dataset_range"].replace({True: "2-std", False: "all"})

# Sort dataframe by recall score.
mse_recall.sort_values(["recall_score", "model_name"], ascending=False, inplace=True)

# Export dataframe.
mse_recall.to_csv(os.getcwd() + os.sep + "beer_output" + os.sep + "metrics" + os.sep + "mse_recall.csv", index=False)

#### 5.1.1.5 Plot graph of recall score and MSE of each model

In [ ]:
# Plot MSE and the recall score of each model to compare metrics.
fig, ax = plt.subplots()
sns.lineplot(x="model_name", y="mse", data=mse_recall, label="MSE", linestyle="dotted", marker="o")
sns.lineplot(x="model_name", y="recall_score", data=mse_recall, label="Recall", linestyle="dashed", marker="D")
ax.set(title="MSE And Recall Score Of Models", xlabel="", ylabel="MSE and recall")
ax.set_xticks(np.arange(len(mse_recall["model_name"])))
plt.xticks(rotation=30, horizontalalignment="right", verticalalignment="top")
ax.legend()
plt.show()
# MSE is lowest on models with maximum recall score, which is to be expected.
# However, MSE doesn't show a reversed pattern of recall. This proves that lower MSE doesn't necessarily mean a better model.

#### 5.1.1.6 Plot graph of recall scores split by model type and dataset range

In [ ]:
# Plot recall scores. Use dodge parameter to create even space between bars when using hue.
# Graph split by model type.
sns.barplot(x="model_name", y="recall_score", data=mse_recall, hue=mse_recall["model_type"], dodge=False)
plt.xticks(rotation=30, horizontalalignment="right", verticalalignment="top")
plt.ylabel(mse_recall["recall_score"].name.capitalize())
plt.title("recall of models split by type".title())
plt.legend(loc="upper right")
plt.show()
# Graph shows that DRF and GBM have the similar results for the same dataset, except for beer full range dataset where DRF performed better.
# Will use DRF models for predictions on the unseen dataset.

# Graph split by dataset range.
sns.barplot(x="model_name", y="recall_score", data=mse_recall, hue=mse_recall["dataset_range"], dodge=False)
plt.xticks(rotation=30, horizontalalignment="right", verticalalignment="top")
plt.ylabel(mse_recall["recall_score"].name.capitalize())
plt.title("recall of models split by dataset range".title())
plt.legend(loc="upper right")
plt.show()
# Graph shows that full range datasets generated better models than datasets with 2*std away ratings.
# This was to be expected because the goal of the project is to predict outliers, not the bulk of average ratings.
# Will keep outliers when testing the model on the unseen beer dataset.

### 5.1.2 Approved Models

In [ ]:
# Print MSE and recall dataframe.
print(mse_recall)

# Approved models (see discussion in Final Report):
# alc_free_all_drf for unseen alcohol-free beer
# light_all_drf for unseen light beer
# regular_all_drf for unseen regular beer

## 5.2 Review Process

In [ ]:
# Create function to plot false negatives split by occurrence of observations in subgroups.

def plot_occurrences(false_negatives_dictionary):
    """Subplots of false negatives split by occurrence in each dataset.

    Args:
    false_negatives_dictionary (dict): Dictionary of false negatives dataframes and their names.

    Returns:
    Seaborn countplot in subplots of false negatives.
    """

    # Create figure and set its title.
    fig = plt.figure()
    fig.suptitle("Occurrence of observations in false negatives".title())
    # Start index of subplots.
    i=0

    # Iterate through input dictionary.
    for key, value in false_negatives_dictionary.items():
        # Access name of dataframe.
        name = str(key)

        # Add subplots sequentially.
        # Mark the first subplot as i+1 because subplot indices start at 1, and i is initialized at 0.
        ax = fig.add_subplot(3, 4, i+1)
        # Set title of subplot.
        ax.set_title(f"{name}", fontsize=7, verticalalignment="top", y=0.9)
        
        # Select rows of each subgroup.
        too_few = value[value["occurrence"] == "too_few"]
        enough = value[value["occurrence"] == "enough"]

        # If both subgroups are present use whole dataframe for plotting, otherwise use the single series.
        # This helps when aligning labels.
        if len(too_few["occurrence"]) > 0:
            sns.countplot(x="occurrence", data=value, order=value["occurrence"].value_counts().index)
        else:
            sns.countplot(x="occurrence", data=enough)

        # Count bars to be plotted because it helps with setting bar width.
        bar_number = value["occurrence"].nunique()

        # Set bar width. Matplotlib divides the plot area into bars according to the number of bars. 
        # The float parameter provided by user is not used as a constant.
        # That's why setting a parameter dependent on number of bars cancels the division made by matplotlib.
        # This results in bars with the same width.
        for patch in ax.patches:
            patch.set_width(0.3*bar_number)
        # Show value counts of observations as labels on top of bars.
        ax.bar_label(ax.containers[0])

        # Set only y label to show they are counts.
        # An x label would crowd the figure because of limited space between rows of graphs.
        # The title of the whole figure already says what's on x axis.
        ax.set(xlabel="", ylabel="Count")

        # Calculate middle points of bars and use them to mark location of x-ticks.
        midpoints = [patch.get_x() + patch.get_width() / 2 for patch in ax.patches]
        ax.set_xticks(midpoints)
        # Define list of labels and set them under x-ticks.
        list_labels = list(value["occurrence"].value_counts().index)
        ax.set_xticklabels(list_labels, fontsize=7)

        # Set common y limit for all subplots in order to have the same scale when visualizing them.
        plt.ylim([0, 20])

        # Increment the index for the next subplot.
        i += 1
    
    # Show all graphs in one figure.
    plt.show()

plot_occurrences(false_negatives)

# Graphs show that most of the wrong predictions were generated for subgroups that had enough observations.

# STEP 6. DEPLOYMENT

## 6.1. Plan Deployment

### 6.1.1 Import and clean unseen file following the same steps as with seen data

In [ ]:
# Import unseen file.
beer_unseen_raw = pd.read_csv(os.getcwd() + os.sep + "unseen_data" + os.sep + "unseen_input" + os.sep + "beer_unseen_raw.csv", header=0)

# Standardize appeareance. Convert column labels to lowercase.
beer_unseen_raw.columns = beer_unseen_raw.columns.str.lower()

# Convert columns values to lowercase if they are strings.
beer_unseen = beer_unseen_raw.applymap(lambda col:col.lower() if type(col) == str else col)

# Convert Name column from object to string.
beer_unseen["name"] = beer_unseen["name"].astype("string")

# Cut alcohol content from end of name and store as separate column.
beer_unseen["abv"] = [name.rsplit(maxsplit=1)[-1] for name in beer_unseen["name"]]

# Convert alcohol content to float.
beer_unseen["abv"] = beer_unseen["abv"].astype(float)

# Validation of uniqueness in beer names. Check for duplicates, remove if found.
beer_unseen.drop_duplicates(subset="name", keep="last", inplace=True)

# Create column for Pasteurization status.
beer_unseen["pasteurization"] = beer_unseen["name"].str.contains("|".join(unpasteurized_words))
beer_unseen["pasteurization"] = beer_unseen["pasteurization"].replace({True: "unpasteurized", False: "pasteurized"})

### 6.1.2 Create datasets based on alcohol content

In [ ]:
# Split unseen dataset into alcohol-free, light and regular beer.
alc_free_unseen = beer_unseen[beer_unseen["abv"] <= 0.5]
light_unseen = beer_unseen[(beer_unseen["abv"] > 0.5) & (beer_unseen["abv"] <= 3)]
regular_unseen = beer_unseen[beer_unseen["abv"] > 3]

# Create list of unseen datasets, list of identificators and zip them.
clean_unseen = [alc_free_unseen, light_unseen, regular_unseen]
unseen_names = ["alc_free_unseen", "light_unseen", "regular_unseen"]
unseen_dict = dict(zip(unseen_names, clean_unseen))

# Export unseen datasets after cleaning.
unseen_clean_path = os.getcwd() + os.sep + "unseen_data" + os.sep + "unseen_output" + os.sep + "unseen_clean"

export_dict_as_csv(unseen_dict, unseen_clean_path)

### 6.1.3 Apply ML models on unseen data

In [ ]:
# Select unseen clean files.
unseen_clean_files = glob.glob(unseen_clean_path + os.sep + "*.csv")

# Import unseen datasets as dictionary into H2O.
unseen_frames = files_to_h2o_frames(unseen_clean_files, "rating")

In [ ]:
# Write function to test models on unseen datasets.

def test_model_on_unseen(frames, pred_output_folder):
    """Apply already trained model to an unseen dataset and output dictionary of dataset names and predictions.
    
    Args:
    frames (dict): Dictionary containing unseen frames.
    pred_output_folder (str): Path to folder where to export predictions.

    Returns:
    csv file with predictions of each model.
    dict: Dictionary of pandas dataframes containing predictions.
    
    """

    # Create dictionary to store names of unseen datasets and their predictions.
    unseen_prediction_dict = {}

    # Iterate through dictionary and store its elements under short variable names to help with readability.
    for key, value in frames.items():
        # Access the name.
        unseen_name = str(key) + "_pred"

        # Access the frame.
        unseen_frame = frames[key]["frame"]

        # Import models.
        if "alc_free" in unseen_name:
            imported_model = h2o.import_mojo(os.getcwd() + os.sep + "beer_output" + os.sep + "models" + os.sep + "drf_models" + os.sep + "alc_free_all_drf.zip")

        elif "light" in unseen_name:
            imported_model = h2o.import_mojo(os.getcwd() + os.sep + "beer_output" + os.sep + "models" + os.sep + "drf_models" + os.sep + "light_all_drf.zip")

        elif "regular" in unseen_name:
            imported_model = h2o.import_mojo(os.getcwd() + os.sep + "beer_output" + os.sep + "models" + os.sep + "drf_models" + os.sep + "regular_all_drf.zip")

        else:
            imported_model = h2o.import_mojo(os.getcwd() + os.sep + "beer_output" + os.sep + "models" + os.sep + "drf_models" + os.sep + "beer_all_drf.zip")
        
        # Generate predictions.
        unseen_prediction = imported_model.predict(unseen_frame)

        # Frames plus predictions.
        unseen_plus_pred = unseen_frame.cbind(unseen_prediction)

        # Convert frames to dataframes.
        unseen_pred_df = unseen_plus_pred.as_data_frame()

        # Append dictionary with prediction.
        unseen_prediction_dict[unseen_name] = unseen_pred_df

        # Export predictions dataframes.
        unseen_pred_df.to_csv(pred_output_folder+ os.sep + unseen_name + ".csv", index=False)

    return unseen_prediction_dict

### 6.1.4 Generate and export predictions

In [ ]:
# Path to store predictions from unseen datasets.
unseen_predictions_path = os.getcwd() + os.sep + "unseen_data" + os.sep + "unseen_output" + os.sep + "unseen_predictions"

# Call function to generate and export predictions on unseen datasets.
unseen_predictions = test_model_on_unseen(unseen_frames, unseen_predictions_path)

# In alc_free and light there is a Warning:
# Test/Validation column “Style” has levels not trained on alt and “Flavor” has levels not trained on herb.
# This means that the training frame the model was built upon had no alt style and no herb flavor.

### 6.1.5 Select and export false negatives

In [ ]:
# Select false negatives from unseen datasets.
unseen_false_negatives = select_false_negatives(unseen_predictions, "rating", "predict", 5)

# Path to store false negatives from unseen datasets.
unseen_false_negatives_path = os.getcwd() + os.sep + "unseen_data" + os.sep + "unseen_output" + os.sep + "unseen_false_negatives"

# Export false negatives from unseen datasets.
export_dict_as_csv(unseen_false_negatives, unseen_false_negatives_path)

### 6.1.6 Calculate and export recall scores on unseen datasets

In [ ]:
# Calculate recall score of models on unseen datasets.
unseen_recall = calculate_recall_score(unseen_predictions, "rating", "predict", 5)

# Sort unseen_recall by recall score.
unseen_recall.sort_values("recall_score", ascending=False, inplace=True)

# Path where to store recall of models on unseen datasets.
unseen_recall_path = os.getcwd() + os.sep + "unseen_data" + os.sep + "unseen_output" + os.sep + "unseen_metrics" + os.sep + "recall_unseen.csv"

# Export recall score of models on unseen datasets.
unseen_recall.to_csv(unseen_recall_path, index=False)

### 6.1.7 Evaluate results

In [ ]:
# Print recall dataframe.
print(unseen_recall)

# Plot recall scores on unseen datasets.
ax = sns.barplot(x="model_name", y="recall_score", data=unseen_recall)
ax.set(xlabel="Model name", ylabel="Recall", title="recall of predictions in unseen datasets".title())
ax.set_ylim([0, 1])
ax.bar_label(ax.containers[0])
plt.show()

# Graph shows that regular beer is the only one from which low rated beer was detected.
# Alcohol-free and light beers either had a scoze of 0, or didn't have low rated beer in the dataset.
# It's possible to improve these scores or lack of score by collecting more data.

In [ ]:
# Print total number of rows and ratings lower than 5 in each unseen dataset to bring context for recall score.

for key, value in unseen_predictions.items():
    # Check number of total rows.
    total_rows = value.shape[0]
    # Check number of low ratings.
    lower_than_five = value[value["rating"] < 5]
    only_lows = lower_than_five.shape[0]
    # Check number of detected low ratings.
    true_positives = value[(value["rating"] < 5) & (value["predict"] < 5)]
    detected = true_positives.shape[0]
    # Check number of undetected low ratings.
    false_negatives = value[(value["rating"] < 5) & (value["predict"] >= 5)]
    undetected = false_negatives.shape[0]
    # Print all counts from above.
    print(f"Dataset: {key} \nRows: {total_rows} \nNumber of low ratings: {only_lows}, out of which: \nDetected: {detected} \nUndetected: {undetected}\n")